In [33]:
%matplotlib inline


Transfer Learning 
==========================


-  **Finetuning the convnet**: Instead of random initializaion, we
   initialize the network with a pretrained network, like the one that is
   trained on imagenet 1000 dataset. Rest of the training looks as
   usual.
-  **ConvNet as fixed feature extractor**: Here, we will freeze the weights
   for all of the network except that of the final fully connected
   layer. This last fully connected layer is replaced with a new one
   with random weights and only this layer is trained.




In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os

plt.ion()   # interactive mode

Load Data
---------

We will use torchvision and torch.utils.data packages for loading the
data.

The problem we're going to solve today is to train a model to classify
**ants** and **bees**. We have about 120 training images each for ants and bees.
There are 75 validation images for each class. Usually, this is a very
small dataset to generalize upon, if trained from scratch. Since we
are using transfer learning, we should be able to generalize reasonably
well.

This dataset is a very small subset of imagenet.

.. Note ::
   Download the data from
   `here <https://download.pytorch.org/tutorial/hymenoptera_data.zip>`_
   and extract it to the current directory.



In [35]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'products'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()



Visualize a few images
^^^^^^^^^^^^^^^^^^^^^^
Let's visualize a few training images so as to understand the data
augmentations.



In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

Training the model
------------------

Now, let's write a general function to train a model. Here, we will
illustrate:

-  Scheduling the learning rate
-  Saving the best model

In the following, parameter ``scheduler`` is an LR scheduler object from
``torch.optim.lr_scheduler``.



In [43]:
log = []

mode = 'finetuning'

file = open("{}/val_loss.txt".format(mode),"w") 
file.write('Loss\n')
file.close()
file = open("{}/val_accuracy.txt".format(mode),"w") 
file.write('Accuracy\n')
file.close()
file = open("{}/train_loss.txt".format(mode),"w") 
file.write('Loss\n')
file.close()
file = open("{}/train_accuracy.txt".format(mode),"w") 
file.write('Accuracy\n')
file.close()


def save_loss(epoch, phase, epoch_loss, epoch_acc):
    file = open("{}/{}_loss.txt".format(mode,phase),"a") 
    file.write("{}\n".format(epoch_loss))
    file.close()
    file = open("{}/{}_accuracy.txt".format(mode,phase),"a") 
    file.write("{}\n".format(epoch_acc))
    file.close()


    

In [44]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            save_loss(epoch, phase, epoch_loss, epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

Visualizing the model predictions
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Generic function to display predictions for a few images




In [45]:
def visualize_model(model, num_images=6):
    images_so_far = 0
    fig = plt.figure()

    for i, data in enumerate(dataloaders['val']):
        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)

        for j in range(inputs.size()[0]):
            images_so_far += 1
            ax = plt.subplot(num_images//2, 2, images_so_far)
            ax.axis('off')
            ax.set_title('predicted: {}'.format(class_names[preds[j]]))
            imshow(inputs.cpu().data[j])

            if images_so_far == num_images:
                return

Finetuning the convnet
----------------------

Load a pretrained model and reset final fully connected layer.




In [46]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Train and evaluate
^^^^^^^^^^^^^^^^^^

It should take around 15-25 min on CPU. On GPU though, it takes less than a
minute.




In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

file.close() 


Epoch 0/24
----------
train Loss: 0.0787 Acc: 0.8509
val Loss: 0.0033 Acc: 1.0000

Epoch 1/24
----------
train Loss: 0.0852 Acc: 0.8696
val Loss: 0.0024 Acc: 1.0000

Epoch 2/24
----------
train Loss: 0.1199 Acc: 0.8385
val Loss: 0.0243 Acc: 0.9167

Epoch 3/24
----------
train Loss: 0.1361 Acc: 0.7764
val Loss: 0.0038 Acc: 1.0000

Epoch 4/24
----------
train Loss: 0.0588 Acc: 0.9006
val Loss: 0.0126 Acc: 0.9722

Epoch 5/24
----------
train Loss: 0.0907 Acc: 0.8571
val Loss: 0.0015 Acc: 1.0000

Epoch 6/24
----------
train Loss: 0.0465 Acc: 0.9379
val Loss: 0.0002 Acc: 1.0000

Epoch 7/24
----------
train Loss: 0.0765 Acc: 0.8882
val Loss: 0.0003 Acc: 1.0000

Epoch 8/24
----------
train Loss: 0.0272 Acc: 0.9689
val Loss: 0.0004 Acc: 1.0000

Epoch 9/24
----------
train Loss: 0.0279 Acc: 0.9565
val Loss: 0.0003 Acc: 1.0000

Epoch 10/24
----------
train Loss: 0.0825 Acc: 0.9130
val Loss: 0.0002 Acc: 1.0000

Epoch 11/24
----------
train Loss: 0.0468 Acc: 0.9503
val Loss: 0.0002 Acc: 1.0000

Ep

In [ ]:
visualize_model(model_ft)

ConvNet as fixed feature extractor
----------------------------------

Here, we need to freeze all the network except the final layer. We need
to set ``requires_grad == False`` to freeze the parameters so that the
gradients are not computed in ``backward()``.

You can read more about this in the documentation
`here <http://pytorch.org/docs/notes/autograd.html#excluding-subgraphs-from-backward>`__.




In [41]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

if use_gpu:
    model_conv = model_conv.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

Train and evaluate
^^^^^^^^^^^^^^^^^^

On CPU this will take about half the time compared to previous scenario.
This is expected as gradients don't need to be computed for most of the
network. However, forward does need to be computed.




In [42]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=20)

Epoch 0/19
----------
train Loss: 0.1155 Acc: 0.7640
val Loss: 0.0174 Acc: 0.9722

Epoch 1/19
----------
train Loss: 0.0674 Acc: 0.9068
val Loss: 0.0083 Acc: 1.0000

Epoch 2/19
----------
train Loss: 0.1100 Acc: 0.8199
val Loss: 0.0323 Acc: 0.9167

Epoch 3/19
----------
train Loss: 0.0662 Acc: 0.9255
val Loss: 0.0164 Acc: 0.9722

Epoch 4/19
----------
train Loss: 0.0636 Acc: 0.9130
val Loss: 0.0580 Acc: 0.9167

Epoch 5/19
----------
train Loss: 0.0745 Acc: 0.8944
val Loss: 0.0099 Acc: 1.0000

Epoch 6/19
----------
train Loss: 0.0970 Acc: 0.8447
val Loss: 0.0101 Acc: 1.0000

Epoch 7/19
----------
train Loss: 0.0930 Acc: 0.8075
val Loss: 0.0029 Acc: 1.0000

Epoch 8/19
----------
train Loss: 0.0555 Acc: 0.9006
val Loss: 0.0089 Acc: 1.0000

Epoch 9/19
----------
train Loss: 0.0647 Acc: 0.9006
val Loss: 0.0047 Acc: 1.0000

Epoch 10/19
----------
train Loss: 0.0670 Acc: 0.8634
val Loss: 0.0056 Acc: 1.0000

Epoch 11/19
----------
train Loss: 0.0709 Acc: 0.8634
val Loss: 0.0080 Acc: 1.0000

Ep

In [ ]:
visualize_model(model_conv)

plt.ioff()
plt.show()